In [2]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle

In [3]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 3.4MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-cp36-none-any.whl size=73269 sha256=c1f0f5ec6c69bd0c681e505a8c4c81ddab3d6d046596f083157f5979ee2d6f78
  Stored in directory: /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


In [4]:
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [6]:
!kaggle competitions download -c home-credit-default-risk

 98% 677M/688M [00:05<00:00, 113MB/s]
100% 688M/688M [00:05<00:00, 127MB/s]


In [7]:
!unzip home-credit-default-risk.zip

Archive:  home-credit-default-risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


Все скачали, пора делать

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [9]:
train = pd.read_csv('application_train.csv')
test = pd.read_csv('application_test.csv')
bureau = pd.read_csv('bureau.csv')
submission = pd.read_csv('sample_submission.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
credit_card_balance = pd.read_csv('credit_card_balance.csv')
prev_app = pd.read_csv('previous_application.csv')

In [29]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder,StandardScaler
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn import tree
import seaborn as sns

In [11]:
num_features = train.select_dtypes(exclude = ["object"]).columns
cat_features = train.select_dtypes(include = ["object"]).columns

In [13]:
most_freq_cat_nans = []
for i in cat_features:
  most_freq_cat_nans.append(train[i].value_counts().idxmax())

In [14]:
for i in range(len(cat_features)):
    train[cat_features[i]].fillna(most_freq_cat_nans[i], inplace=True)
for i in num_features:
    train[i].fillna(train[i].median(), inplace=True)

In [15]:
train.isna().sum().max()

0

In [16]:
y = train['TARGET']

In [17]:
train = train.drop('TARGET', axis=1)
train = train.drop('SK_ID_CURR', axis=1)

In [18]:
test_ids = test['SK_ID_CURR']
test = test.drop('SK_ID_CURR', axis=1)

In [19]:
num_features = test.select_dtypes(exclude = ["object"]).columns
cat_features = test.select_dtypes(include = ["object"]).columns

In [21]:
most_freq_cat_nans_test = []
for i in cat_features:
    most_freq_cat_nans_test.append(test[i].value_counts().idxmax())

In [23]:
for i in range(len(cat_features)):
    test[cat_features[i]].fillna(most_freq_cat_nans[i], inplace=True)
for i in num_features:
    test[i].fillna(test[i].median(), inplace=True)

In [25]:
train_size = train.shape[0]
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data = pd.get_dummies(all_data, columns = cat_features, dummy_na = True)
train_data = all_data[:train_size]
test_data = all_data[train_size:]

Делаем дерево решений

In [33]:
model_tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=6, min_samples_leaf=2)
model_tree.fit(train_data, y)
predict = model_tree.predict_proba(test_data)[:,1]
submission['TARGET'] = predict
submission.to_csv('submission.csv', index = None)

In [35]:
!kaggle competitions submit -c home-credit-default-risk -f submission.csv -m "DecisionTreeClassifier давай работай"

100% 1.26M/1.26M [00:00<00:00, 5.82MB/s]
Successfully submitted to Home Credit Default Risk

score 0.69655

Ищем параметры чтоб поднять скор

In [ ]:
params_tree = {'criterion':('gini', 'entropy'),
          'max_depth':[6, 7, 8],
          'min_samples_leaf':[1, 2]}
model_tree = tree.DecisionTreeClassifier()
clf_tree_GS = GridSearchCV(model_tree, params_tree, scoring ='roc_auc', cv=5)
clf_tree_GS.fit(train_data, y)
clf_tree_GS.best_params_

In [31]:
model_tree = tree.DecisionTreeClassifier(criterion='gini', max_depth=8, min_samples_leaf=1)
model_tree.fit(train_data, y)
predict = model_tree.predict_proba(test_data)[:,1]
submission['TARGET'] = predict
submission.to_csv('submission.csv', index = None)

In [ ]:
!kaggle competitions submit -c home-credit-default-risk -f submission.csv -m "DecisionTreeClassifier давай работай"

После GRID_SearchCV лучшие параметры для дерева
1. критерий 'gini',
2. максимальная глубина : 8, 
3. 'min_samples_leaf': 1


После этих параметров **ROC AUC** 0.70398 что немного лучше чем вариант с другими параметрами